In [1]:
import tensorflow as tf
import pickle
import numpy as np
import math as math
import matplotlib.pyplot as plt
import os
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

C:\tools\Anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [2]:
# Opening file for reading in binary mode
with open('./data/data7.pickle', 'rb') as f:
    data = pickle.load(f, encoding='latin1')  # dictionary type

In [3]:
# Preparing y_train and y_validation for using in Keras
# data['y_train'] = tf.keras.utils.to_categorical(data['y_train'], num_classes=43)
# data['y_validation'] = tf.keras.utils.to_categorical(data['y_validation'], num_classes=43)

In [4]:
y_train = data['y_train']
y_val = data['y_validation']
y_test = data['y_test']

In [5]:
# Making channels come at the end
X_train = data['x_train'] = data['x_train'].transpose(0, 2, 3, 1)
X_val = data['x_validation'] = data['x_validation'].transpose(0, 2, 3, 1)
X_test = data['x_test'] = data['x_test'].transpose(0, 2, 3, 1)

In [6]:
# Showing loaded data from file
for i, j in data.items():
    if i == 'labels':
        print(i + ':', len(j))
    else:
        print(i + ':', j.shape)

x_train: (86989, 32, 32, 1)
y_test: (12630,)
x_validation: (4410, 32, 32, 1)
y_validation: (4410,)
labels: 43
y_train: (86989,)
x_test: (12630, 32, 32, 1)


In [7]:
def create_model(activation='relu', dropout=0.0, optimizer='adam', neurons=64, channel=1): 
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(neurons, kernel_size=3, padding='same', activation=activation, input_shape=(32, 32, channel)))
    model.add(tf.keras.layers.MaxPool2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(dropout))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation=activation))
    model.add(tf.keras.layers.Dense(256, activation=activation))
    model.add(tf.keras.layers.Dense(43, activation='softmax'))
  
    model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, 
                optimizer=tf.keras.optimizers.Adam(), 
                metrics=['accuracy'])

    model.summary()

    return model

In [8]:
def create_model2(image_shape=(32, 32, 3)):

    vgg16 = tf.keras.applications.VGG16(include_top=True,
                                        weights=None,
                                        input_shape=image_shape,
                                        classes=43)

    model = tf.keras.Sequential()

    # Freeze the layers
    for layer in model.layers:
        layer.trainable = False
    
    model.add(vgg16)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(512, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.4))

    # Add 'softmax' instead of earlier 'prediction' layer.
    model.add(tf.keras.layers.Dense(43, activation='softmax'))

    model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, 
                optimizer=tf.keras.optimizers.Adam(), 
                metrics=['accuracy'])

    model.summary()

    return model

In [9]:
model = create_model()
EPOCHS = 1
BATCH_SIZE = 512

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               4194560   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
____

In [11]:
with tf.device('/device:GPU:0'):

    # Random Rotations
    # define data preparation
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20, brightness_range=[0.3, 0.6])
    datagen.fit(X_train)
    
    
    history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                                      validation_data=(X_val, y_val),
                                      steps_per_epoch=len(X_train) // BATCH_SIZE,
                                      epochs=EPOCHS)
            

169/169 [==============================] - 30s 179ms/step - loss: 7.7600 - acc: 0.4811 - val_loss: 3.6513 - val_acc: 0.1073


In [12]:
predictions = model.predict(X_test, batch_size=512)

y_pred = [np.argmax(prediction) for prediction in predictions]

acc = accuracy_score(y_test, y_pred)
pre = precision_score(y_test, y_pred, average="macro")
rec = recall_score(y_test, y_pred, average="macro")
f1 = f1_score(y_test, y_pred, average="macro")

C:\tools\Anaconda3\envs\tf\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\tools\Anaconda3\envs\tf\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
print(acc, pre, rec, f1)

0.09144893111638955 0.28657854974229585 0.09012097642835762 0.07442758281890982
